<a href="https://colab.research.google.com/github/rahulsinghsss/All-DataStructure-in-one-go/blob/main/STEELEYE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3
import requests
import xml.etree.ElementTree as ET
import zipfile
import csv
from io import BytesIO
from zipfile import ZipFile
import boto3

# Step 1: Download the XML file from the link
xml_url = 'https://registers.esma.europa.eu/solr/esma_registers_firds_files/select?q=*&fq=publication_date:%5B2022-01-03T00:00:00Z+TO+2022-01-03T23:59:59Z%5D&wt=xml&indent=true&start=0&rows=100'
response = requests.get(xml_url)
xml_content = response.content

# Step 2: Parse through the XML file to find the first download link with file_type=DITINS
root = ET.fromstring(xml_content)
download_link = None
for doc in root.iter('doc'):
    if doc.find('str[@name="file_type"]').text == 'DLTINS':
        download_link = doc.find('str[@name="download_link"]').text
        break

# Step 3: Download the zip file from the extracted download link
zip_content = requests.get(download_link).content

# Step 4: Extract the XML file from the zip file
zip_file = ZipFile(BytesIO(zip_content))
xml_file = zip_file.open(zip_file.namelist()[0])
xml_content = xml_file.read()

# Step 5: Parse the contents of the XML file and extract the required data
root = ET.fromstring(xml_content)
rows = []
for row in root.findall(".//{*}FinInstrmGnlAttrbts"):
    id = row.find("{*}Id").text
    full_name = row.find("{*}FullNm").text
    clssfctn_tp = row.find("{*}ClssfctnTp").text
    cmmdty_deriv_ind = row.find("{*}CmmdtyDerivInd").text
    ntnl_ccy = row.find("{*}NtnlCcy").text
    issr_element = row.find("{*}Issr")
    issr = issr_element.text if issr_element is not None else None
    rows.append([id, full_name, clssfctn_tp, cmmdty_deriv_ind, ntnl_ccy, issr])

# Step 6: Write the extracted data into a CSV file with the given header
header = ['FinInstrmGnlAttrbts.Id', 'FinInstrmGnlAttrbts.FullNm', 'FinInstrmGnlAttrbts.ClssfctnTp', 'FinInstrmGnlAttrbts.CmmdtyDerivInd', 'FinInstrmGnlAttrbts.NtnlCcy', 'Issr']
with open('output.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(header)
    writer.writerows(rows)

#Downolad the file
from google.colab import files
files.download('output.csv')

#Uplading to s3 bucket--------------------------------------------------------------------------------

# Set your AWS credentials
aws_access_key_id = 'your_access_key_id'
aws_secret_access_key = 'your_secret_access_key'

# Set the name of your S3 bucket and the file path of your CSV file
bucket_name = 'your_bucket_name'
file_path = '/path/to/your/file.csv'

# Create an S3 client
s3_client = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

# Upload the CSV file to S3
with open(file_path, 'rb') as file:
    s3_client.upload_fileobj(file, bucket_name, 'your_filename.csv')

print('File uploaded successfully')
